# Введение в MapReduce модель на Python


In [3]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [4]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [5]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [6]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

In [7]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [8]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [10]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [11]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [12]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [13]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [14]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [15]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [16]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [17]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.4726556497075234),
 (1, 1.4726556497075234),
 (2, 1.4726556497075234),
 (3, 1.4726556497075234),
 (4, 1.4726556497075234)]

## Inverted index 

In [18]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [19]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [20]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [21]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('a', 2), ('what', 10)])]

## TeraSort

In [22]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.00080821586145019),
   (None, 0.019284317217848068),
   (None, 0.031173996648541924),
   (None, 0.04136026003674609),
   (None, 0.0683254220417675),
   (None, 0.24530715817986626),
   (None, 0.2710475162125673),
   (None, 0.2761083534269546),
   (None, 0.3538704289641068),
   (None, 0.3674202686000735),
   (None, 0.40756767410010686),
   (None, 0.4079278656820513),
   (None, 0.41489565158408015),
   (None, 0.44407635767881914),
   (None, 0.4499681614159988),
   (None, 0.4967897129086287)]),
 (1,
  [(None, 0.5207655403853846),
   (None, 0.5328516944678419),
   (None, 0.5967209363138527),
   (None, 0.6001391082066033),
   (None, 0.6552122130188338),
   (None, 0.6642658959330666),
   (None, 0.7046395649958402),
   (None, 0.802346531176548),
   (None, 0.8200630451209431),
   (None, 0.8214242619122328),
   (None, 0.8516382504091402),
   (None, 0.9404681026633106),
   (None, 0.9594654488596593),
   (None, 0.99093316288953)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [34]:
def MAP_MAX(_, row: NamedTuple):
    yield ("max", row)

# Поиск самого старшего пользователя
def REDUCE_MAX(max_str: str, rows: Iterator[NamedTuple]):
    max_age = 0
    max_age_user = None
    for row in rows:
        if row.age > max_age:
            max_age = row.age
            max_age_user = row
    yield (max_str, max_age_user)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output

[('max', User(id=0, age=55, social_contacts=20, gender='male'))]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [37]:
def MAP_AVG(_, row: NamedTuple):
    yield ("average age", row)

# Подсчет среднего значения возраста
def REDUCE_AVG(average_str: str, rows: Iterator[NamedTuple]):
    count = 0
    summa = 0
    for row in rows:
      summa += row.age
      count += 1
    yield (average_str, summa/count)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
output

[('average age', 34.5)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [39]:
def groupByKey(iterable):
    # Сортировка по ключу
    iterable = sorted(iterable, key=lambda user: user[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
print(f"max: {output}")

output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
print(f"average: {output}")

max: [('max', User(id=0, age=55, social_contacts=20, gender='male'))]
average: [('average age', 34.5)]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [43]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=54, gender="male", social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=32, gender="female", social_contacts=753),
    User(id=4, age=21, gender="male", social_contacts=100),
]

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection) / maps)) # распределяем по вычислительным узлам
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

# группировка пользователей по id
def MAP_UNIQUE(_, user: str):
    yield (user.id, user)

# оставляем только первого среди одинаковых id
def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

7 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=55, social_contacts=20, gender='male'),
   User(id=2, age=25, social_contacts=500, gender='female'),
   User(id=4, age=21, social_contacts=100, gender='male')]),
 (1,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [44]:
# предикатом C является проверка параметра age. Можно взять любой, например пол
def MAP_SELECT(_, row: NamedTuple):
    if row.age == 21:
        yield (row, row)

def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=4, age=21, social_contacts=100, gender='male'),
  [User(id=4, age=21, social_contacts=100, gender='male')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [53]:
input_collection = [
    dict(id=0, age=54, gender="male", social_contacts=20),
    dict(id=0, age=25, gender="female", social_contacts=240),
    dict(id=1, age=27, gender="male", social_contacts=642),
    dict(id=2, age=16, gender="male", social_contacts=123),
    dict(id=2, age=35, gender="female", social_contacts=247),
    dict(id=3, age=32, gender="male", social_contacts=521),
    dict(id=3, age=32, gender="male", social_contacts=753),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

def MAP_PROJECTION(_, iterable): # делаем проекцию по S
  S = ["age", "gender"]
  filtered_data = [{key: item[key] for key in S if key in item} for item in input_collection]
  for data in filtered_data:
    yield (data["age"], data)

def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]): # убираем повторения
  unique_rows = set()
  for row_data in rows:
      row_tuple = tuple(row_data.items())
      unique_rows.add(row_tuple)

  return [{key: value for key, value in row_tuple} for row_tuple in unique_rows]

def RECORDREADER():
    return [(u["id"], u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[{'age': 54, 'gender': 'male'},
 {'age': 25, 'gender': 'female'},
 {'age': 27, 'gender': 'male'},
 {'age': 16, 'gender': 'male'},
 {'age': 35, 'gender': 'female'},
 {'age': 32, 'gender': 'male'}]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [55]:
input_collection_a = [
    User(id=1, age=54, gender="male", social_contacts=20),
    User(id=2, age=25, gender="female", social_contacts=240),
    User(id=3, age=27, gender="male", social_contacts=642),
    User(id=4, age=16, gender="male", social_contacts=123),
    User(id=5, age=35, gender="female", social_contacts=247),
]

input_collection_b = [
    User(id=5, age=35, gender="female", social_contacts=247),
    User(id=6, age=31, gender="male", social_contacts=521),
    User(id=7, age=32, gender="female", social_contacts=753),
]

def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)

# для каждого пользователя создается пара (t, t), даже если он есть только в одной из выборок
def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    for elem in rows:
      yield(row, elem)

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(1, User(id=1, age=54, social_contacts=20, gender='male')),
 (2, User(id=2, age=25, social_contacts=240, gender='female')),
 (3, User(id=3, age=27, social_contacts=642, gender='male')),
 (4, User(id=4, age=16, social_contacts=123, gender='male')),
 (5, User(id=5, age=35, social_contacts=247, gender='female')),
 (5, User(id=5, age=35, social_contacts=247, gender='female')),
 (6, User(id=6, age=31, social_contacts=521, gender='male')),
 (7, User(id=7, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [57]:
def MAP_INTERSECTION(_, row: NamedTuple):
    yield(row.id, row)
    
# Возвращает пользователя, только если он присутствует в обоих выборках
def REDUCE_INTERSECTION(row: str, rows: Iterator[NamedTuple]):
  if len(rows) ==2:
    yield(rows[0], rows[1])

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[(User(id=5, age=35, social_contacts=247, gender='female'),
  User(id=5, age=35, social_contacts=247, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [60]:
# группировка по пользователям
def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)

# возвращаем пользователей, входящих только в первую выборку (и не входящих во вторую)
def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)

def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]

output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=1, age=54, social_contacts=20, gender='male'),
 User(id=2, age=25, social_contacts=240, gender='female'),
 User(id=3, age=27, social_contacts=642, gender='male'),
 User(id=4, age=16, social_contacts=123, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [66]:
class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str
    pet_id: int

users_collection = [
    User(id=1, age=54, gender="male", social_contacts=20, pet_id=1),
    User(id=2, age=25, gender="female", social_contacts=240, pet_id=1),
    User(id=3, age=27, gender="male", social_contacts=642, pet_id=2),
    User(id=4, age=16, gender="male", social_contacts=123, pet_id=3),
    User(id=5, age=35, gender="female", social_contacts=247, pet_id=4),
]

class Pet(NamedTuple):
    id: int
    name: str

pets_collection = [
    Pet(id=1, name="cat"),
    Pet(id=2, name="dog"),
    Pet(id=3, name="parrot"),
    Pet(id=4, name="crocodile"),
]

# группировка пользователей по id животного
def MAP_JOIN(pet_id, row):
    yield (pet_id, row)

def REDUCE_JOIN(pet_id, rows):
    users = []
    pet = None

    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            pet = row

    for row in rows:
        if type(row) is User:
            # Возвращает тройки (a,b,c), где a - пользователь, b - id питомца, c - вид питомца
            yield (row, row.pet_id, pet)


# На первом месте стоит id животного, на втором - сам питомец или пользователь
def RECORDREADER():
    return [(user.pet_id, user) for user in users_collection] + [
        (pet.id, pet) for pet in pets_collection
    ]


output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=1, age=54, social_contacts=20, gender='male', pet_id=1),
  1,
  Pet(id=1, name='cat')),
 (User(id=2, age=25, social_contacts=240, gender='female', pet_id=1),
  1,
  Pet(id=1, name='cat')),
 (User(id=3, age=27, social_contacts=642, gender='male', pet_id=2),
  2,
  Pet(id=2, name='dog')),
 (User(id=4, age=16, social_contacts=123, gender='male', pet_id=3),
  3,
  Pet(id=3, name='parrot')),
 (User(id=5, age=35, social_contacts=247, gender='female', pet_id=4),
  4,
  Pet(id=4, name='crocodile'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [68]:
# группировка по id животного
def MAP_GROUP(pet_id, user, pet):
    yield (pet_id, user)

# подсчет количества владельцев каждого животного
def REDUCE_GROUP(pet_id, rows):
  yield f"pet with id = {pet_id} has {len(rows)} owner(s)"

def RECORDREADER():
    return [(pet_id, user, city) for user, pet_id, city in joined_data]

output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['pet with id = 1 has 2 owner(s)',
 'pet with id = 2 has 1 owner(s)',
 'pet with id = 3 has 1 owner(s)',
 'pet with id = 4 has 1 owner(s)']

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [72]:
from typing import Iterator, Tuple

input_matrix = [
    (1, 0, 2), (1, 3, 3), (0, 2, 3),
    (1, 0, 0), (1, 1, 0), (1, 0, 1),
    (2, 0, 8), (0, 1, 1), (5, 0, 3),
]

input_vector = [
    (0, 1), (1, 2), (2, 3)
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    grouped = {}
    for key, value in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )

def MAP_MATRIX_VECTOR(_, matrix_row):
    row, col, value = matrix_row

    for vec_col, vec_value in input_vector:
        if vec_col == col:
            yield (row, value * vec_value)

def REDUCE_MATRIX_VECTOR(row, values: Iterator[int]):
    yield (row, sum(values))

def RECORDREADER():
    return [(None, m) for m in input_matrix]

output = MapReduce(RECORDREADER, MAP_MATRIX_VECTOR, REDUCE_MATRIX_VECTOR)
output = list(output)
output

[(1, 3), (0, 11), (2, 8), (5, 3)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [44]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [73]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  for i in range(I):
        k2 = (i, k)
        v2 = small_mat[i, j] * w
        yield (k2, v2)
  
def REDUCE(key, values):
    (i, k) = key
    # solution code that yield(k3,v3) pairs
    k3 = (i, k)
    v3 = sum(values)
    yield (k3, v3)

Проверьте своё решение

In [74]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [75]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [76]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

# генерируем пары элементов обеих матриц
def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

# перемножаем полученные элементы двух матриц
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)

# суммируем полученные произведения для вычисления элементов результирующей матрицы
def REDUCE(key, values):
    (i, k) = key
    # solution code that yield(k3,v3) pairs
    k3 = (i, k)
    v3 = sum(values)
    yield (k3, v3)

In [77]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution))  # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [78]:
maps = 2
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    # разделение между вычислительными узлами происходит по строкам первой матрицы
    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))

# возвращает произведения соответствующих элементов из обеих матриц, группируя их по индексу в результирующей матрице
def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)


def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = sum(values)

    yield (k3, v3)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]

# соединение полученных произведений с разных вычислительных узлов
solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true
     

240 key-value pairs were sent over a network.
[[1.12638062 0.24445752 0.33709441 0.94447359 1.59676428 0.8213588
  0.28935557 0.59977793 0.44372835 1.47941384 0.82762411 1.06390492
  0.89538672 0.88430165 0.59294699 0.74438812 1.07168806 0.2063368
  0.82563357 1.04646571 1.29522355 0.53718869 1.39374603 0.90297908
  1.20868822 1.12425302 0.79732564 0.85547536 1.13127801 0.39929691
  0.8405452  1.64749409 0.82068846 0.32397786 0.91875233 0.67992142
  1.17358121 0.80419208 0.96094974 0.67724946]
 [0.77844518 0.32603931 0.20942421 0.85061329 1.10904399 0.72711133
  0.28559226 0.28216163 0.27699053 1.22933185 0.69679122 0.78104192
  0.85883242 0.40899651 0.52481964 0.66251888 0.98975095 0.22622474
  0.8327872  0.68590205 0.75619241 0.69940707 1.0712296  1.02146553
  0.95759508 0.83921092 0.64400338 0.58367892 1.06352605 0.31066545
  0.8055355  1.29254453 0.35538505 0.48696547 0.55378534 0.35493132
  0.73153724 0.97671251 0.46635047 0.26031161]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Да, решение будет работать, потому что каждая пара значений, создаваемая RECORDREADER-ом, идентифицируется по ее индексам.